In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 
import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

def deEmojify(text):
    # Strip all non-ASCII characters to remove emoji characters
    
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None
    
class MyStreamListener(tweepy.StreamListener):
    # Override tweepy.StreamListener to add logic to on_status
    
    def clean_tweet(self, tweet):
        #Cleans tweet text by removing links, special characters - using regex statements.
        
        if tweet:
            return ' '.join(re.sub("(@[^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        else:
            return None
        

    def on_status(self, status):
        #Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text=deEmojify(status.text)
        sentiment = TextBlob(self.clean_tweet(text)).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=credentials.mysqlpasswd,
    database="twitterdb",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [ ]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @restartleader: … Iran’s regime would have been changed by now!
This shows that they are all being ruled by one specific group!

👉 Liste…
Long: None, Lati: None
@Symonsez13 @MingGao26 Kerry and O advise Iran as we tweet. THAT is treason! You are misinformed. Research. Read. Bye bye.
Long: None, Lati: None
RT @kurdistannews24: @ciyager_ypg video 2
uprising against #Assad in #DeirEzzor #Syria. (Day 2)
(Regime control area) 
Protesters demanding…
Long: None, Lati: None
Filmmaker Bahman Ghobadi's mother released after a few hours of detention, passport confiscated [fa] https://t.co/Cgt7nCEinG #Iran
Long: None, Lati: None
RT @AliAbunimah: 1 million Iraqis and Iranians died after the Iraqi invasion of Iran and an 8-year war where Iraq was funded by the Gulf re…
Long: None, Lati: None
RT @charliekirk11: Iran continues to execute gay people because as they say: “moral principles”

Yet western media &amp; the left do everything…
Long: None, Lati: None
FIFA tells Iran women need to be allowed

RT @restartiran23: @realDonaldTrump 🔴MAKE IRAN GREAT AGAIN!
https://t.co/ZFmWIZ7mto
Long: None, Lati: None
"As for Iran attacking Saudi Arabia.....that hasn't actually been demonstrated to be true, and…" — Lonesome_Coyote https://t.co/ZhttBAWLX4
Long: None, Lati: None
Very unsurprising that Liz Cheney is calling for a war with Iran. She's just like her demented dad... if there's a… https://t.co/2Didz2Czkb
Long: None, Lati: None
PM Modi should have warned China also indirectly in #Houstan in the presence of @realDonaldTrump  Even Iran too.… https://t.co/ONrInWvQzN
Long: None, Lati: None
RT @Jali_Cat: ‼️Former members of the Obama administration are holding back channel communications with Iran, advising how to defeat USA fo…
Long: None, Lati: None
Iran Releases Seized British Oil Tanker Stena Impero: Report https://t.co/1I9rzZSDSX
Long: None, Lati: None
RT @ShamaJunejo: When we talk about diplomacy...
The sizes of red carpet clears it all. We got a door mate, and Modi had Trump in their

RT @AltAntifa: @LoriinUtah They are paid by USA's foreign enemies (Russia, China, Iran, etc) to foment dissent and to promote a destructive…
Long: None, Lati: None
RT @suzannedimaggio: Every senior member of Trump's national security team advised him to carry out military strikes against Iran. Not a si…
Long: None, Lati: None
RT @cgtnamerica: Iran's Foreign Minister @JZarif discusses the future of the #Irandeal or #JCPOA with CGTN's @asiehnamdar. He reveals that…
Long: None, Lati: None
RT @karolcummins: 🌳Breathe2🌳

Iran warns foreign forces to stay out of Gulf, amid new US deployment

Schiff: ‘We very well may have crossed…
Long: None, Lati: None
RT @iran_policy: #IRAN: Suppressive Bases and Pictures of Khamenei and Khomeini Set Ablaze in 17 Cities https://t.co/TDG3ElK0P7
#IranProtes…
Long: None, Lati: None
RT @sarahleah1: This is the same #Saudi foreign minister who gave us “outrageous, outlandish and frankly laughable” lies about his governme…
Long: None, Lati: None
RT @PeterSweden7:

RT @B52Malmet: I’m old enough to remember when North Korea was threatening to bomb Guam and we were worried about the end of the world. Now…
Long: None, Lati: None
RT @DionNissenbaum: NEW: In apparent effort to distance themselves from Iran, Houthi fighters warn Tehran is planning follow-on attacks to…
Long: None, Lati: None
RT @DearAuntCrabby: Liz Cheney urges attack on Iran over oil

Question: Is that all the Cheney family knows is war?

Answer: Of course, WHY…
Long: None, Lati: None
RT @GarethPorter: Now it turns out it could take many months to repair the facilities damaged by the missile attack -- not the 10 weeks the…
Long: None, Lati: None
RT @thehill: President Trump: "I would like to see Iran be a great and very wealthy country because it has tremendous capability. It has tr…
Long: None, Lati: None
RT @happypeaceful3: @America1stTiger How stupid can John Kerry be? well here it is, he sound like a nut. So the best way to deal with Iran…
Long: None, Lati: None
RT @peymaneh123: T

RT @LeeCamp: Always ask questions about the mainstream narratives forced on you!

Is Iran a threat to the US? Did Epstein commit suicide? C…
Long: None, Lati: None
RT @vasudha_ET: This sexualisation of women as unwrapped objects that need protection is so deeply offensive to all of us. We only need to…
Long: None, Lati: None
RT @carterwpage: Excellent discussions today @LindseyGrahamSC, @DevinNunes and @MariaBartiromo.

I received $0 from Russia during the Obama…
Long: None, Lati: None
RT @TrueEyeTheSpy: Well said, @SecPompeo!
We support the People of Iran.
Our goal is not to destroy their economy. We are slowly increasing…
Long: None, Lati: None
RT @4AllSoulKind: 1.  Aussie PM State Visit 
2. Collapsed Iran’s Financial Sytem.
3. Assisted Allies in flattening Iran’s Syrian our post.…
Long: None, Lati: None
Saudi Arabia seeks action against Iran after oil attack, allies wary
 https://t.co/gD1zcRaRFQ
Long: None, Lati: None
@CNN Good thing I didn’t book their deluxe Middle East package to

...

RT @NickSotoudeh: #AtenaDaemi was serving a 7-YEAR prison sentence in # Iran for advocating children's rights. Now she's been slapped with…
Long: None, Lati: None
RT @deenie7940: Pompeo: New Sanctions Will Deny Iran ‘The Resources To Foment The Exact Kind Of Strikes We’ve Seen’ | The Daily Caller http…
Long: None, Lati: None
RT @MashhourII: @IlhanMN The pro-Iran islamist @mehdirhasan who is deceiving idiots by pretending to be liberal, is putting aside his islam…
Long: None, Lati: None
RT @IranLionness: They should remove all the charges against all women who have been sentenced for trying to enter into stadiums.

Let's se…
Long: None, Lati: None
RT @thehill: President Trump: "I would like to see Iran be a great and very wealthy country because it has tremendous capability. It has tr…
Long: None, Lati: None
RT @Shnrch33: This is why we need borders &amp; fixed immigration laws!Anybody who chants death to America, death to Israel needs no access to…
Long: None, Lati: None
RT @inthecopa:

RT @JessieJaneDuff: Iran's in a corner.

President Trump prefers peaceful methods over violent conflict.

However, if action is necessary,…
Long: None, Lati: None
RT @robjh1: The left and liberal media want war so that they will have another fake reason to impeach the president. And as if Congress wou…
Long: None, Lati: None
RT @sahouraxo: #Iran didn’t invade Iraq on a lie and butcher millions.

Iran didn’t bomb Libya and pave the way for open air slave markets.…
Long: None, Lati: None
Iran to lay out Gulf security proposal at UN general assembly https://t.co/jMhvR05Z55
Long: None, Lati: None
The USA just sold Saudi Arabia Tons of weapons when Trump took office. I think Saudi Arabia can protect themselves… https://t.co/JNlflKuwy0
Long: None, Lati: None
RT @LeeCamp: Always ask questions about the mainstream narratives forced on you!

Is Iran a threat to the US? Did Epstein commit suicide? C…
Long: None, Lati: None
RT @thehill: President Trump: "Nothing's ever off the table, completely, 

RT @Josephine4Trump: @Miraculously111 And Carrie’s daughter is married to a high ranking Iranians son. Hence the Iran scam.
Long: None, Lati: None
RT @RozitaRiazati: BREAKING - President of World Football's governing body, FIFA, says his organisation been assured by #Iran that women wi…
Long: None, Lati: None
RT @RL9631: In Iran, Saudi Arabia, Somalia, blasphemy is punishable by death.
This Imam says that according to the Shari'a, Christmas celeb…
Long: None, Lati: None
RT @ChloeSalsameda: #NEW: Former Defense Secretary Gen. James Mattis:

Iran is "continuing to do what they've done for nearly four decades…
Long: None, Lati: None
‘Not on my watch’: Corbyn would veto a US strike on Iran, ally claims https://t.co/Hf0JTV0XkN
Long: None, Lati: None
RT @BaddCompani: This is Why
https://t.co/5Sjbu5QmOY
Long: None, Lati: None
What will USA do to Iran is a test for its policy on China.but I doubt those masochism manias who expect USA to occ… https://t.co/cZxPUcQX01
Long: None, Lati: None
RT @t

@realDonaldTrump In the below video, the representative of the leader of Iran, officially announces that Restartees… https://t.co/J9uuVokYXU
Long: None, Lati: None
RT @indypremium: Is Trump taking the US to war with Iran? https://t.co/suDUrLnVtz
Long: None, Lati: None
Iran asks West to leave Persian Gulf as tensions heightened - ABC News https://t.co/ID1AbJMLBJ
Long: None, Lati: None
RT @restartleader: Below video with more than 1m views got deleted by Twitter.
With credible proof, we do have millions of downloads in Tel…
Long: None, Lati: None
this is the nation that  destroyed  twin towers and MORE 
Saudi were all taught to fiy plane in building and leard… https://t.co/75TpVO8V64
Long: None, Lati: None
RT @peymaneh123: Maryam Rajavi: Rise up to continue the uprisings and liberate Iran from the mullahs’ occupation-Message to Iranian student…
Long: None, Lati: None
RT @morphonios: And alleged Iran-backed drone strike that didn't injure a single person has resulted in U.S. troops being 

Insane And Ill-Advised: Trump's Future War With Iran, Part 2 | Zero Hedge https://t.co/6lk46fec5z
Long: None, Lati: None
WHAT?
Long: None, Lati: None
'Not on my watch': Corbyn would veto a US strike on Iran, ally claims https://t.co/fzWByt1s2m https://t.co/i0XBoe3QPe
Long: None, Lati: None
RT @JohnBrennan: Iran struck Saudi oil facilities because of sanctions—does he seriously expect more sanctions to deter them? 

It should b…
Long: None, Lati: None
RT @LeeCamp: Always ask questions about the mainstream narratives forced on you!

Is Iran a threat to the US? Did Epstein commit suicide? C…
Long: None, Lati: None
@paulsperry_ Remember John Kerry going to Iran for the sole purpose  of undermining  the USA foreign policy. The MS… https://t.co/2cSwrGltNt
Long: None, Lati: None
RT @B52Malmet: I’m old enough to remember when North Korea was threatening to bomb Guam and we were worried about the end of the world. Now…
Long: None, Lati: None
@AnsarAAbbasi 2/3 LNG Case:
"The Qatar LNG deal has a